# 合并相邻同类型 messages

In [7]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass(
        prompt="Enter your OpenAI API key (required if using OpenAI): "
    )

base_url = os.environ.get("BASE_URL")
model_name = os.environ.get("MODEL_NAME")
deepseek_model_name = os.environ.get("DEEPSEEK_MODEL_NAME")

print(base_url, model_name)

https://dashscope.aliyuncs.com/compatible-mode/v1 qwen-plus


In [8]:
from langchain_deepseek import ChatDeepSeek



llm = ChatDeepSeek(
    model=deepseek_model_name,
    temperature=0,
    max_retries=2,
    base_url=base_url,
)

In [13]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    merge_message_runs,
)

messages = [
    SystemMessage("you're a good assistant."),
    SystemMessage("you always respond with a joke."),
    HumanMessage([{"type": "text", "text": "i wonder why it's called langchain"}]),
    HumanMessage("and who is harrison chasing anyways"),
    AIMessage(
        'Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'
    ),
    AIMessage("Why, he's probably chasing after the last cup of coffee in the office!"),
]

merged = merge_message_runs(messages)
llm.invoke(merged)
print("\n\n".join([repr(x) for x in merged]))

SystemMessage(content="you're a good assistant.\nyou always respond with a joke.", additional_kwargs={}, response_metadata={})

HumanMessage(content=[{'type': 'text', 'text': "i wonder why it's called langchain"}, 'and who is harrison chasing anyways'], additional_kwargs={}, response_metadata={})

AIMessage(content='Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!\nWhy, he\'s probably chasing after the last cup of coffee in the office!', additional_kwargs={}, response_metadata={})


从结果来看相邻的同类型消息被和合并成了一条

In [14]:

from langchain_anthropic import ChatAnthropic

# llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)
# Notice we don't pass in messages. This creates
# a RunnableLambda that takes messages as input
merger = merge_message_runs()
chain = merger | llm
chain.invoke(messages)

AIMessage(content=[], additional_kwargs={}, response_metadata={'id': 'msg_01EKxRiF3Tc9rA4fZDYfaE3Y', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 84, 'output_tokens': 3, 'server_tool_use': None}, 'model_name': 'claude-3-sonnet-20240229'}, id='run--e9683e1a-5aec-4605-a0f0-ba023280df0c-0', usage_metadata={'input_tokens': 84, 'output_tokens': 3, 'total_tokens': 87, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})